# Fiber Study
Wastyk et al. (2021) from the Sonnenburg Lab showed in a feeding study the inter-individual differences in inflammatory immune response after intervention with a high-fiber diet. Here, we use microbial sequencing data from this study to compare the results of MICOM metabolic modeling with the results obtained by the authors. Specifically, we are curious whether the SCFA production profiles from our metabolic models agree with the inflammatory immune response group used by the authors to categorize study participants

In [ ]:
import pandas as pd
import micom
import micom.measures
from plotnine import *
import scipy

%matplotlib inline

## Taxonomy Table
Load up the taxonomic abundances and build a taxonomy table for use in MICOM

In [ ]:
taxonomy = pd.read_csv('../data/fiberstudy.csv') # Load abundance data
taxonomy = pd.melt(taxonomy,# Melt into a longform dataframe
                   id_vars = 'sample_id',
                   value_vars = taxonomy.columns[1:-1], 
                   var_name = 'id',
                   value_name = 'abundance')
taxonomy['genus'] = taxonomy['id']
taxonomy.id.unique()

## Import Metadata

In [ ]:
metadata = pd.read_csv('../data/raw_data/fiber_immune_groups.csv', index_col = 1)
metadata['Immune_group'] = metadata['Immune_group'].str.replace(
              'group1','HighInflammation').str.replace(
              'group2','LowInflammationI').str.replace(
              'group3','LowInflammationII').astype('str')
metadata

## Build Models
Now, we'll build our models, with cutoff of 0.001

In [ ]:
models = micom.workflows.build(taxonomy,
                               out_folder = '../models/fiber_study',
                               model_db = '../agora103_genus.qza',
                               cutoff = 0.001, 
                               threads = 10)

## Load Medium
Load the high fiber medium we'll use for growth

In [ ]:
medium = pd.read_csv('../media/highfiber.csv')

## Grow Models
Next, grow the samples with the high fiber medium and a tradeoff value of 0.7

In [ ]:
manifest = pd.read_csv('../models/fiber_study/manifest.csv') # Read the model manifest
growth = micom.workflows.grow(manifest,
                              '../models/fiber_study',
                              medium=medium,
                              tradeoff=.7,
                              strategy='none',
                              threads = 20)

## Calculate predicted SCFA production
Specifically, we'll look at butyrate and propioante production

In [ ]:
production = micom.measures.production_rates(growth) # Calculate production
scfa = production[(production['metabolite']=='but[e]')|(production['metabolite']=='ppa[e]')] # Isolate butyrate and propoionate
scfa = pd.pivot(scfa,
                index = 'sample_id',
                columns = 'metabolite',
                values = 'flux') # Pivot data
scfa['timepoint'] = scfa.index.str.split('_').str[1].astype('int')
scfa.index = scfa.index.str.split('_').str[0].astype('int')
scfa['immune_group'] = scfa.index.map(metadata['Immune_group'].to_dict()) # Add immune group metadata
scfa.dropna(inplace = True)
scfa

## Plot Results
Now, plot the predicted production among each inflammation group, at each timepoint across the intervention and average

In [ ]:
plt1 = (ggplot(scfa,
    aes(x = 'timepoint', y = 'but[e]'))
    +geom_point(color = 'cadetblue')
    +geom_smooth(method = 'lm', linetype = '--')
    +labs(x = 'Timepoint', y = 'Predicted Butyrate \n ($\dfrac{mmol}{gDW*h}$)')
    +facet_wrap(facets = 'immune_group')
    +theme(text = element_text(size=20),panel_background=element_rect(fill = "white",
                                colour = "white",size = 0.5, linetype = "solid"),panel_grid= element_blank(),
                                axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                                axis_line_y = element_blank(), axis_text_x=element_text(rotation = 20, hjust = 1)))
plt1

In [ ]:
ggsave(plt1, '../results/fiber_but_timepoint.svg',dpi = 300, format = 'svg')

In [ ]:
plt2 = (ggplot(scfa,
    aes(x = 'timepoint', y = 'ppa[e]'))
    +geom_point(color = 'cadetblue')
    +geom_smooth(method = 'lm', linetype = '--')
    +labs(x = 'Timepoint', y = 'Predicted Propionate \n ($\dfrac{mmol}{gDW*h}$)')
    +facet_wrap(facets = 'immune_group')
    +theme(text = element_text(size=20),panel_background=element_rect(fill = "white",
                                colour = "white",size = 0.5, linetype = "solid"),panel_grid= element_blank(),
                                axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                                axis_line_y = element_blank(), axis_text_x=element_text(rotation = 20, hjust = 1)))
plt2

In [ ]:
ggsave(plt2, '../results/fiber_ppa_timepoint.svg',dpi = 300, format = 'svg')

Let's also look at average production across the entire intervention per immune group.

In [ ]:
plt3 = (ggplot(scfa, 
        aes(x = 'immune_group', y = 'but[e]'))
        +geom_boxplot(fill = 'cadetblue',size = 1, width = 0.4)
        +labs(x = 'Immune Response Group', y = 'Predicted Butyrate ($\dfrac{mmol}{L*h}$)')
        +scale_x_discrete(limits = ['HighInflammation','LowInflammationI','LowInflammationII'],
                          labels = ['High \n Inf.', 'Low \n Inf. I', 'Low \n Inf. II'])
        +theme(text = element_text(size=20, color = 'black'),panel_background=element_rect(fill = "white",
                                colour = "white",size = 0.5, linetype = "solid"),panel_grid= element_blank(),
                                axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                                legend_position='right',axis_text_x=element_text(rotation = 20, hjust = 1)))
plt3

In [ ]:
ggsave(plt3, '../results/fiber_but_total.svg',dpi = 300, format = 'svg')

In [ ]:
plt4 = (ggplot(scfa,
        aes(x = 'immune_group', y = 'ppa[e]'))
        +geom_boxplot(fill = 'cadetblue',size = 1, width = 0.4)
        +labs(x = 'Immune Response Group', y = 'Predicted Propionate($\dfrac{mmol}{L*h}$)')
        +scale_x_discrete(limits = ['HighInflammation','LowInflammationI','LowInflammationII'],
                          labels = ['High \n Inf.', 'Low \n Inf. I', 'Low \n Inf. II'])
        +theme(text = element_text(size=20, color = 'black'),panel_background=element_rect(fill = "white",
                                colour = "white",size = 0.5, linetype = "solid"),panel_grid= element_blank(),
                                axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                                legend_position='right',axis_text_x=element_text(rotation = 20, hjust = 1)))
plt4

In [ ]:
ggsave(plt4, '../results/fiber_ppa_total.svg',dpi = 300, format = 'svg')

In [ ]:
scipy.stats.mannwhitneyu(scfa[scfa['immune_group'] == 'HighInflammation']['but[e]'], scfa[scfa['immune_group'] == 'LowInflammationI']['but[e]'])

## Save Results

In [ ]:
scfa.to_csv('../scfa_SourceData_Fig4.csv')